In [1]:
import pandas as pd

# Hypothetical Households
income = [50e3, 100e3, 500e3]
household_size = [1, 1, 1]
working = [1, 1, 1]

df = pd.DataFrame({
        'income': income,  # Market income
        'working': working,  # Number of working adults
        'household_size': household_size,  # Total household size
    })



In [2]:
import numpy as np

def net_income(income, rate, deduction):
    # Use numpy's maximum for a vectorized max function
    liability = np.maximum(income - deduction, 0) * rate
    return income - liability

In [ ]:
import monte_carlo as m

# no standard deduction
df = df.assign(
    net_income = net_income(df['income'], 0.2, 0) 
)

model = m.generate_ols_model(df)
sigma_hat = m.get_sigma_hat(model)
tau_hat = (1 - model.params.iloc[1])

print([round(tau_hat, 10), sigma_hat])

[np.float64(-0.0), 0.0]


In [ ]:
# 10k standard deduction
df = df.assign(
    net_income = net_income(df['income'], 0.2, 10e3)
)

model = m.generate_ols_model(df)
sigma_hat = m.get_sigma_hat(model)
tau_hat = (1 - model.params.iloc[1])

print([tau_hat, sigma_hat])

[np.float64(0.017829081065672936), 0.005013190075270371]


In [ ]:
# 20k standard deduction
df = df.assign(
    net_income = net_income(df['income'], 0.2, 20e3)
)

model = m.generate_ols_model(df)
sigma_hat = m.get_sigma_hat(model)
tau_hat = (1 - model.params.iloc[1])

print([tau_hat, sigma_hat])

[np.float64(0.034792345166639205), 0.009567333254769203]


In [11]:
def net_income_mtr(income, deduction, rates, lower_brackets):
    # Use numpy's maximum for a vectorized max function
    agi = np.maximum(income - deduction, 0)
    liability = np.zeros_like(agi)
    
    for i in range(len(rates)):
        # For each income bracket, calculate tax on the portion of income that falls in this bracket
        if i < len(rates) - 1:  # Not the highest bracket
            # Amount of income in this bracket (capped at the next bracket boundary)
            bracket_income = np.minimum(
                np.maximum(agi - lower_brackets[i], 0),
                lower_brackets[i + 1] - lower_brackets[i]
            )
            liability += bracket_income * rates[i]
        else:  # Highest bracket
            # Amount of income in the highest bracket
            bracket_income = np.maximum(agi - lower_brackets[i], 0)
            liability += bracket_income * rates[i]
    return income - liability

In [17]:
rates = [0.2, 0.3]
brackets = [0, 200e3]
df = df.assign(
    net_income = net_income_mtr(df['income'], 20e3, rates, brackets)
)

model = m.generate_ols_model(df)
sigma_hat = m.get_sigma_hat(model)
tau_hat = (1 - model.params.iloc[1])

print([tau_hat, sigma_hat])

[np.float64(0.06835211545034225), 0.0003664267058930104]
